In [16]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

#Get list of teams
week1 = pd.read_csv('Week1.csv')
teams = set(week1['Team'])

#Create empty table
table = DataFrame(columns=[['H','H','H','H','H','A','A','A','A','A'],
                         ['W','L','D','F','A','W','L','D','F','A']],
                  index=teams)
table = table.fillna(0)

#Get list of fixtures
fixtures = pd.read_csv('Fixtures.csv')
fixtures = fixtures.set_index('Week')

#For each week so far
for w in range(1,17):
    
    #Load those stats and sum goals and own goals by team into a new dataframe
    stats = pd.read_csv('week%s.csv'%w)
    goals = stats[['Team','Player','G','GON']]
    goal_totals = goals.groupby(['Team']).sum()

    #Get that week's home and away teams
    home = list(fixtures.ix[w,'Home'])
    away = list(fixtures.ix[w,'Away'])

    #For each fixture
    for x in range(0,len(home)):
        
        #Add own goals into the goals total for home and away teams
        hg = goal_totals['G'][home[x]] + goal_totals['GON'][away[x]]
        ag = goal_totals['G'][away[x]] + goal_totals['GON'][home[x]]
        
        #Update the goals for and against
        table['H','F'][home[x]] += hg
        table['A','A'][away[x]] += hg
        table['A','F'][away[x]] += ag
        table['H','A'][home[x]] += ag
        
        #Update WLD records
        if hg>ag :
            table['H','W'][home[x]] += 1
            table['A','L'][away[x]] += 1
        elif hg==ag :
            table['H','D'][home[x]] += 1
            table['A','D'][away[x]] += 1
        else :
            table['H','L'][home[x]] += 1
            table['A','W'][away[x]] += 1
            
#Add in totals GD and points
table['T','F'] = table['H','F'] + table['A','F']
table['T','A'] = table['H','A'] + table['A','A']
table['T','GD'] = table['T','F'] - table['T','A']
table['T','Pts'] = 3*(table['H','W'] + table['A','W']) + table['H','D'] + table['A','D']

#Sort according to points, GD, GS
table.sort_values([('T','Pts'),('T','GD'),('T','F')], ascending=[False,False,False], inplace=True)
    
print table

    

      H                A                 T            
      W  L  D   F   A  W  L  D   F   A   F   A  GD Pts
LEIC  5  1  2  16  13  5  0  3  18   9  34  22  12  35
ARS   4  1  2  11   5  6  2  1  18   8  29  13  16  33
MCI   7  2  0  25  11  3  2  2   7   6  32  17  15  32
MUN   4  0  3   9   1  4  3  2  12  11  21  12   9  29
TOT   4  1  4  15   7  2  1  4  11   7  26  14  12  26
CRY   4  4  1  12   9  4  2  1   9   6  21  15   6  26
WAT   3  3  2   6   6  4  2  2  12  10  18  16   2  25
WHU   2  2  4  12  11  4  2  2  13  10  25  21   4  24
LIV   3  2  3  10  11  3  2  3  10   8  20  19   1  24
EVE   3  2  3  17  12  2  1  5  12   9  29  21   8  23
STK   3  3  1   7   7  3  2  4   6   7  13  14  -1  23
SOU   3  3  2  13  11  2  2  4   8   8  21  19   2  21
WBA   2  4  2  10  14  3  2  3   6   7  16  21  -5  20
BOUR  2  3  3  10  13  2  5  1  10  18  20  31 -11  16
NEW   2  3  3  13  12  2  5  1   5  19  18  31 -13  16
CHE   3  4  1  10  11  1  5  2   8  15  18  26  -8  15
SWA   2  3